# Instance Segmentation evaluation

Test a Detector on A Customized Dataset (e.g., Mask RCNN)

Notice: MMDetection **only support evaluating mask AP of dataset in COCO format for now**. Other methods and more advanced usages can be found in the [doc](https://mmdetection.readthedocs.io/en/latest/tutorials/customize_dataset.html). So we need to reorganize the dataset into a COCO format firstly.

## (1) Test on a test dataset (with labeled data)

In [ ]:
# Single-gpu testing

# "--work-dir" saves the predicted results
# "--out" saves the test results in a "pkl" file, this file can be used to predict images or output confusion matrix (see below)
# "--show-dir" saves the predicted images in test dataset

!python tools/test.py \
    /scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper_Peng/Yolact_r101/lr_0.001/Pollutant_yolact_r101.py \
    /scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper_Peng/Yolact_r101/lr_0.001/best_coco_segm_mAP_50_epoch_27.pth \
    --work-dir /scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper_Peng/Yolact_r101/lr_0.001/test/ \
    --out /scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper_Peng/Yolact_r101/lr_0.001/test/result.pkl \
    --cfg-options test_evaluator.classwise=True
    
    # --cfg-options \
    # --show-dir /scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper2_exp5/Oos_20per/FRCNN_Frozen_4_PreTrain_GV/pred_images_in_test_dataset/

## (2) Predict images in a folder

Note: inference_detector only supports single-image inference for now

In [ ]:
import os
from mmdet.apis import init_detector, inference_detector
from mmdet.utils import register_all_modules
from mmdet.registry import VISUALIZERS
import mmcv
import datetime

# define the model configure and model weight
config_file =r'/scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper_Peng/Cascade_mask_rcnn_r50/lr_0.01/Pollutant_cascade_mask_rcnn_r50.py'
checkpoint_file = r"/scratch/tjian/PythonProject/DP_MMDetection/checkpoints/train_weights/Paper_Peng/Cascade_mask_rcnn_r50/lr_0.01/best_coco_segm_mAP_50_epoch_33.pth"


#Register all modules in mmdet into the registries
register_all_modules()
# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')  # or device='cpu'

# define test image path and output path
folder_path = r"/scratch/tjian/Data/test_Peng/"
out_path = r"/scratch/tjian/Data/test_Peng_pre/"

file_names = os.listdir(folder_path)

starttime = datetime.datetime.now()

for filename in file_names:
        if filename.endswith('.jpg'):
            image_path = os.path.join(folder_path, filename)
            img = mmcv.imread(image_path, channel_order='rgb')
            result = inference_detector(model, img)
            # print(result)
            
            # init the visualizer(execute this block only once)
            visualizer = VISUALIZERS.build(model.cfg.visualizer)
            # the dataset_meta is loaded from the checkpoint and
            # then pass to the model in init_detector
            visualizer.dataset_meta = model.dataset_meta
            
            out_file_name= os.path.join(out_path, filename)
            # show the results
            visualizer.add_datasample('result',
                                      img,
                                      data_sample=result,
                                      draw_gt=False,
                                      wait_time=0,
                                      out_file=out_file_name,
                                      pred_score_thr=0.5
                                      )

endtime = datetime.datetime.now()

# calculate taining time
days = (endtime - starttime).days
seconds = (endtime - starttime).seconds
test_time = round((days*24*60*60+seconds)/60)
print("test time: ", test_time, " min")
            

## (3) Confusion matrix (CM)

(1) The font size of the output confusion matrix can be modified in **tools/analysis_tools/confusion_matrix.py** (Line 175-187)

(2) The default CM output is normalized CM showing the percentage value. I changed it to be the number value (e.g., the number of TP) in **tools/analysis_tools/confusion_matrix.py**

(3) pkl file is generated in the code above (**Test on a test dataset**)

In [ ]:
!python tools/analysis_tools/confusion_matrix.py \
    checkpoints/train_weights/Paper_Peng/Cascade_mask_rcnn_r50/lr_0.01/Pollutant_cascade_mask_rcnn_r50.py \
    checkpoints/train_weights/Paper_Peng/Cascade_mask_rcnn_r50/lr_0.01/test_process/result.pkl \
    checkpoints/train_weights/Paper_Peng/Cascade_mask_rcnn_r50/lr_0.01/test_process/ \
    --show \
    --score-thr 0.5 \
    --tp-iou-thr 0.5
    
# ${CONFIG}  ${DETECTION_RESULTS}  ${SAVE_DIR} --show

## (4) Predict images in a folder and save bbox and segm in an excel file

(Only windows system)

Note:

(1) The function of saving results in excel only works on Windows system 

(2) define excel path in untils.bbox_segm.py

(3) define source image folder below

(4) Two folders will be generated in "--out-dir", including (1) predicted images, and (2) bbox and mask info in json file (mask info is encoded)

(5) I mainly modify the "/mmdet/apis/det_inferencer.py" to realize this function, referring: https://blog.csdn.net/m0_46246301/article/details/130561039



In [ ]:
!python demo/image_demo.py U:/AIMMW/Tianlong/Pollutant/Model_application/pred_img_before_I_am_China/mixture-feeding/ \
    checkpoints/train_weights/Paper_Peng/Cascade_mask_rcnn_r50/lr_0.01/Pollutant_cascade_mask_rcnn_r50.py \
    --weights checkpoints/train_weights/Paper_Peng/Cascade_mask_rcnn_r50/lr_0.01/best_coco_segm_mAP_50_epoch_33.pth \
    --out-dir U:/AIMMW/Tianlong/Pollutant/Model_application/pred_img_before_I_am_China/mixture-feeding_pred/ \
    --pred-score-thr 0.5 \
    --device cpu